In [27]:
import hist
import numpy as np
from plot_utils import adjust_plot
import matplotlib.pyplot as plt
import pickle
from utils_v1 import Histfit, computeJER
from scipy.optimize import curve_fit
import matplotlib.gridspec as gridspec

In [28]:
%load_ext autoreload

%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [29]:
import mplhep as hep
hep.style.use("CMS")

In [30]:
year = 2018
era = "2018"

if era == "2016":
    jerfile = "Summer20UL16_JRV3_MC_PtResolution_AK4PFchs.txt"
    filename = 'samples/flatPU_JMENano_2016.txt'
    year =2016
    
if era == '2016APV':
    jerfile = "Summer20UL16APV_JRV3_MC_PtResolution_AK4PFchs.txt"    
    filename = 'samples/flatPU_JMENano_2016APV.txt'
    year = 2016
if era == '2017':
    jerfile = "Summer20UL17_JRV1_MC_PtResolution_AK4PFchs.txt"
    filename = 'samples/flatPU_JMENano_2017.txt'
    year = 2017
    more = 'JME'
if era == '2018':
    jerfile = "Summer20UL18_JRV1_MC_PtResolution_AK4PFchs.txt"
    filename = 'samples/flatPU_JMENano_2018.txt'
    year = 2018
    more = 'JME'  #change more according to filename

In [31]:
# with open("QCD_pt_response_"+era+more+".pkl", "rb") as f:
#     output = pickle.load( f )

with open("QCD_pt_response_2017JME.pkl", "rb") as f:
    output = pickle.load( f )

In [32]:
hist_0 = output["pt_reco_over_gen"]
hist_0.axes

(StrCategory(['2017'], growth=True, name='dataset', label='Primary dataset'),
 Variable(array([   10.,    11.,    12.,    13.,    14.,    15.,    17.,    20.,
           23.,    27.,    30.,    35.,    40.,    45.,    57.,    72.,
           90.,   120.,   150.,   200.,   300.,   400.,   550.,   750.,
         1000.,  1500.,  2000.,  2500.,  3000.,  3500.,  4000.,  5000.,
        10000.]), name='pt', label='$p_{T}$ [GeV]'),
 Regular(300, 0, 2, name='frac', label='Fraction'),
 Variable([0, 0.261, 0.522, 0.783, 1.044, 1.305, 1.566, 1.74, 1.93, 2.043, 2.172, 2.322, 2.5, 2.65, 2.853, 2.964, 3.139, 3.489, 3.839, 5.191], name='eta', label='$\\eta$'),
 Variable([0, 10, 20, 30, 40, 50, 60, 70, 80], name='pileup', label='$\\mu$'))

In [33]:
datasets = hist_0.axes[0].centers
pt_bins = hist_0.axes[1].edges
frac_bins = hist_0.axes[2].edges
eta_bins = hist_0.axes[3].edges
pileup_bins = hist_0.axes[4].edges
#rho_bins = hist_0.axes[5].edges

pt_values = hist_0.axes[1].centers
frac_values = hist_0.axes[2].centers
eta_values = hist_0.axes[3].centers
pileup_values = hist_0.axes[4].centers
#rho_values = hist_0.axes[5].centers

pt_widths = hist_0.axes[1].widths
frac_widths = hist_0.axes[2].widths
eta_widths = hist_0.axes[3].widths
pileup_widths = hist_0.axes[4].widths
#rho_widths = hist_0.axes[5].widths

In [34]:
def jerfunc(x, p0, p1, p2, p3):
    return np.sqrt(p0*np.abs(p0)/(x*x)+p1*p1*np.power(x,p3) + p2*p2)

In [36]:
#vs pileup (then converted to rho value using the relation derived)

import pandas as pd
#eras = ['2017', '2018']
eras = ['2017']
df_fit = pd.DataFrame({ "eta_low":[], "eta_high":[], "par0":[],"par0_unc":[], "par1":[],"par1_unc":[],
                       "par2":[],"par2_unc":[], "par3":[],"par3_unc":[] , "era":[]})
n_rho_min = 0
n_rho_max = 5
n_eta_min = 0
n_eta_max = 10
n_rho = n_rho_max - n_rho_min
n_eta = n_eta_max - n_eta_min
for era in eras:

    if era == "2016":
        jerfile = "Summer20UL16_JRV3_MC_PtResolution_AK4PFchs.txt"
        filename = 'samples/flatPU_JMENano_2016.txt'
        year =2016
        more = "_premix"
    if era == '2016APV':
        jerfile = "Summer20UL16APV_JRV3_MC_PtResolution_AK4PFchs.txt"    
        filename = 'samples/flatPU_JMENano_2016APV.txt'
        year = 2016
        more = "_premix"
    if era == '2017':
        jerfile = "Summer20UL17_JRV1_MC_PtResolution_AK4PFchs.txt"
        filename = 'samples/flatPU_JMENano_2017.txt'
        year = 2017
        more = "JME"
    if era == '2018':
        jerfile = "Summer20UL18_JRV1_MC_PtResolution_AK4PFchs.txt"
        filename = 'samples/flatPU_JMENano_2018.txt'
        year = 2018
        more = "JME"
        
    with open("QCD_pt_response_"+era+more+".pkl", "rb") as f:
        output = pickle.load( f )
    hist_0 = output["pt_reco_over_gen"]
    
    for i_rho in range(n_rho_min, n_rho_max):
        gs = gridspec.GridSpec(nrows=2, ncols= n_eta, height_ratios=[4, 1], hspace = 0.0)
        for i_eta in range(n_eta_min, n_eta_max):
            hist_rho_pt_frac = hist_0.project("eta","pileup","pt","frac").to_numpy()[0][i_eta]
            hist_pt_frac = hist_rho_pt_frac[i_rho]
            histfit = Histfit(hist_pt_frac, frac_values, pt_values)
            histfit.store_parameters()

            sigma_list = histfit.parameters["sigma"]
            sigma_error_list  = np.array(histfit.parameters["sigmaErr"])
            

            sel = (histfit.parameters["sigma"] != None ) & (pt_values>=30) 

            popt,pcov = curve_fit(jerfunc,  np.array(pt_values[sel], dtype = 'float64'),  np.array(sigma_list[sel], dtype = 'float64') , 
                                  sigma = np.array(sigma_error_list[sel],dtype = 'float64') , p0 = [4.352, 0.2533, 3.106*10**-5, -0.4109],  bounds =([-10,-5,-3,-5],[10,2,2,5]) )    

            df_fit2 = pd.DataFrame({ "eta_low":[eta_bins[i_eta]], "eta_high":[eta_bins[i_eta+1]], 
                                    "par0":[popt[0]],"par0_unc":[np.sqrt(pcov[0][0])], "par1":[popt[1]],"par1_unc":[np.sqrt(pcov[1][1])],
                                    "par2":[popt[2]],"par2_unc":[np.sqrt(pcov[2][2])], "par3":[popt[3]],"par3_unc":[np.sqrt(pcov[3][3])] , "era":[era]})
            df_fit = pd.concat([df_fit,df_fit2])
            
         
            

TypeError: unsupported operand type(s) for ^: 'float' and 'int'

In [24]:
df_fit.head(10)

,eta_low,eta_high,par0,par0_unc,par1,par1_unc,par2,par2_unc,par3,par3_unc,era
0,0.000,0.261,0.000000e+00,0.000000,-0.531724,0.062408,-0.029803,0.001654,-0.773178,0.045135,2017
0,0.261,0.522,0.000000e+00,0.000000,-0.574943,0.069804,-0.032002,0.001423,-0.808915,0.046006,2017
0,0.522,0.783,0.000000e+00,0.000000,-0.519507,0.045994,-0.033486,0.001206,-0.763714,0.034366,2017
0,0.000,0.261,0.000000e+00,0.000000,-0.611308,0.070726,-0.030756,0.001246,-0.822421,0.042885,2017
0,0.261,0.522,0.000000e+00,0.000000,-0.629611,0.060157,0.032079,0.001179,-0.834722,0.036570,2017
0,0.522,0.783,0.000000e+00,0.000000,-0.651541,0.065904,-0.035622,0.001234,-0.847109,0.039373,2017
0,0.000,0.261,0.000000e+00,0.000000,-0.649594,0.063019,-0.031168,0.001203,-0.843469,0.036822,2017
0,0.261,0.522,-4.332140e+00,0.560359,1.088991,0.132842,-0.034731,0.000652,-1.016401,0.041356,2017
0,0.522,0.783,2.040480e-10,0.000000,0.630060,0.064968,-0.034426,0.001271,-0.826058,0.040102,2017
